# Image classification - redux

We load the same data set as last week (only using 3-channel images instead of 1-channel images) to classify lungs as "normal" or with "pneumonia". As before, we augment the dataset, using an `ImageDataGenerator`.

In [ ]:
#pip install opencv-python

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,  GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import confusion_matrix

In [ ]:
labels = ['NORMAL','PNEUMONIA']
img_size = 192
def load_data(data_dir):
    x_data = [] 
    y_data = []
    for label in labels: 
        path = os.path.dirname(os.getcwd())
        path = os.path.join(path+'/Week 7/'+data_dir, label)
        class_num = labels.index(label)
        for img in [f for f in os.listdir(path) if not f.startswith('.')]:
            try:
                img_arr = cv2.imread(os.path.join(path, img))
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                x_data.append(resized_arr)
                y_data.append(class_num)
            except Exception as e:
                print(e)
    x = np.stack(x_data,axis=0).reshape(-1,img_size,img_size,3)
    x = x / 255
    y = np.array(y_data,dtype='float32')    
    return x, y

In [ ]:
x_train, y_train = load_data('chest_xray/train')
x_val, y_val = load_data('chest_xray/val')
x_test, y_test = load_data('chest_xray/test')

In [ ]:
n = 5
plt.figure(figsize=(2*n, 4))
for i in range(n):
    # display normal
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_train[np.where(y_train==0.0)[0][i]],cmap='gray')
    plt.title("normal")
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display pneumonia
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(x_train[np.where(y_train==1.0)[0][i]],cmap='gray')
    plt.title("pneumonia")
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
datagen = ImageDataGenerator(
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True)  # randomly flip images horizontally

datagen.fit(x_train)

## 1. Loading an existing model

Instead of creating our own model, we load `MobileNet` as a basemodel. As you can see in the summary, this is quite the deep CNN! We load the model without its top layer, which is a fully connected layer (`include_top=False`). This is because we want to adapt the network for our specific prediction task.

When loading an existing model from TensorFlow, we can usually specify what parameters (`weights`) we want. Here, we are getting weights that were learned when training the model on the ImageNet (`'imagenet'`) dataset.

In [ ]:
base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=(img_size,img_size,3),
                                            include_top=False,
                                            weights='imagenet')
base_model.summary()

Usually, we keep the weights of the original model as they are. That is, when training with our data, we only adjust the weights of our own layers, that we will add. The easiest is to set the whole original model to be "untrainable", but you could also do this layer-by-layer.

In [ ]:
base_model.trainable = False

Let's take another look at the summary. Focus specifically on the numbers of trainable and non-trainable parameters.

In [ ]:
base_model.summary()

Our model will now combine the `MobileNet` model with three additional layers:
1. a `GlobalAveragePooling2D` layer (which summarizes the first two dimensions into a single value)
1. a (hidden) `Dense` layer with 8 neurons and a `'relu'` activation function, as well as
1. a `Dense` output layer with `'sigmoid'` activation function

In [ ]:
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

We can compile the model as usual, specifying `loss`, `optimizer`, and `metrics`:

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

And we can summarize it. In the summary, the `MobileNet` in total is given as one layer.

In [ ]:
model.summary()

## 2. Training the new top layers

We are now ready to get training. Note that, even if we fixed the parameters of the `MobileNet`, forward- and back-propagation now take a lot longer, since our neural network has gotten a lot deeper. Hence, so does the training process (even though it is still much much faster than when training the entire network from scratch).

In [ ]:
lr_red_callback = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=1e-6)
log = model.fit(datagen.flow(x_train,y_train, batch_size = 32),
                epochs=12,
                validation_data=datagen.flow(x_val,y_val),
                callbacks=[lr_red_callback])

## 3. Evaluating the model

We can now evaluate the model. We went from approx. 90% accuracy on the test set to 92% accuracy. That is quite the difference, when it comes to diagnosing medical images correctly. And this is without any fine-tuning, and using "only" a relatively simple neural network compared to the state of the art.

In [ ]:
model.evaluate(x_test, y_test)